In [85]:
import networkx as nx
import pandas as pd
import scripts.read_graph as rg
import random
import plotly.plotly as py
from plotly.graph_objs import *
import math
import time
import numpy
import os

In [42]:
MG = rg.read('graphs/sp_final.gexf')

In [43]:
def getpath(nested_dict, value, prepath=()):
    for k, v in nested_dict.items():
        path = prepath + (k,)
        if v == value: # found value
            return path
        elif hasattr(v, 'items'): # v is a dict
            p = getpath(v, value, path) # recursive call
            if p is not None:
                return p

In [44]:
def get_distance(lon1, lat1, lon2, lat2):
    
    R = 6373.0

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    
    return distance*1000.0 #em M, por isso multiplica por 1000.0

#0.002 é uma caixa de 200-300m +-
    

In [218]:
class Maps():
    
    def __init__(self, token):
        self.token = 'pk.eyJ1IjoiamFvdHRhIiwiYSI6ImNpdzN3YmVzZjAzeXMydXFsNHRlNGRkZncifQ.6jlHzH9N6kmCXmnDQ0gwrw'
    
    def plotly(self, visited_set, MG):
        
        stops = [MG.node[i] for i in visited_set['stop_id_to'].tolist()]
        COORD = (stops[0]['location']['lat'],stops[0]['location']['long']) # sp coordinates
        
        mapbox_access_token = 'pk.eyJ1IjoiY2hlbHNlYXBsb3RseSIsImEiOiJjaXFqeXVzdDkwMHFrZnRtOGtlMGtwcGs4In0.SLidkdBMEap9POJGIe1eGw' 
        
        data = Data([
            Scattermapbox(
                lat=[s['location']['lat'] for s in stops[1:]],
                lon=[s['location']['long'] for s in stops[1:]],
                mode='markers',
                marker=Marker(
                    size=6,
                    color=['rgb(253, 63, 0)' for i in range(len(stops[1:]))],
                    opacity=0.7
                ),
                hoverinfo='text',
                showlegend=False
            ),
            Scattermapbox(
                lat=[stops[0]['location']['lat']],
                lon=[stops[0]['location']['long'] ],
                mode='markers',
                marker=Marker(
                    size=20,
                    color=['rgb(51, 204, 51)' for i in range(len(stops[0]))],
                    opacity=0.7
                ),
                hoverinfo='skip',
                showlegend=False
            )
        ])
        
        layout = Layout(
            autosize=True,
            hovermode='closest',
            width= 1100,
            height=1000,
            mapbox=dict(
                accesstoken=mapbox_access_token,
                bearing=0,
                center=dict(
                    lat=COORD[0],
                    lon=COORD[1]
                ),
                pitch=0,
                zoom=11
            ),
        )

        fig = dict(data=data, layout=layout)
        return fig
    
    def mapbox(self, visited_set, MG):
        
        stops = [MG.node[i] for i in visited_set['stop_id_to'].tolist()]
        
        
        html = ("""<!DOCTYPE html>
                <html>
                <head>
                <meta charset=utf-8 />
                <title>A simple map</title>
                <meta name='viewport' content='initial-scale=1,maximum-scale=1,user-scalable=no' />
                <script src='https://api.mapbox.com/mapbox.js/v3.0.1/mapbox.js'></script>
                <link href='https://api.mapbox.com/mapbox.js/v3.0.1/mapbox.css' rel='stylesheet' />
                <style>
                  body { margin:0; padding:0; }
                  #map { position:absolute; top:0; bottom:0; width:100%; }
                </style>
                </head>
                <body>

                <div id='map'></div>
                <script>

                L.mapbox.accessToken = 'pk.eyJ1IjoiamFvdHRhIiwiYSI6ImNpdzN3YmVzZjAzeXMydXFsNHRlNGRkZncifQ.6jlHzH9N6kmCXmnDQ0gwrw'\n""")
                
                
        html = html + "var map = L.mapbox.map('map', 'mapbox.streets').setView([{}, {}], 13);\n".format(stops[0]['location']['lat'], stops[0]['location']['long'])
        
        html = html + "L.circle([%s, %s], 200, {color:'orange',stroke:false,fill:true,fillOpacity:0.7}).addTo(map)\n" % (stops[0]['location']['lat'], stops[0]['location']['long'])
        
        html = html + 'var points = [' 
        for i in range(len(stops)):
            if i == 0:
                continue
            elif i == 1:
                html = html + '[{}, {}]'.format(stops[i]['location']['lat'], stops[i]['location']['long']) 
            else:
                html = html + ',[{}, {}]'.format(stops[i]['location']['lat'], stops[i]['location']['long']) 
        
       
        html = (html +       
                """
                ]
                
                points.forEach( function(s) { 
                    L.circle([s[0], s[1]], 30, 
                        {color:'black',
                        stroke:false,
                        fill:true,
                        fillOpacity:0.7}).addTo(map);
                  } );
        
                </script>

                </body>
                </html>
                """)
        with open('map.html', 'w') as f:
            f.write(html)
            
        os.system('python -m webbrowser -t "file:////Users/Admin/Documents/Projects/Kairos/Kairos/map.html"')
        
        

In [222]:
class Walker(Maps):
    
    def __init__(self, graph, stop_id, travel_time, current_time):
        self.MG = graph 
        self.stop_id = stop_id 
        self.total_time_travel = travel_time * 60 # minutos para segundos 
        self.traveled_time = 0 
        self.visited_set = [] 
        self.tentative_value = {} 
        self.start_time = current_time * 60 * 60 # hora para segundo
        self.walking_speed = 1.4 #em m/s
        self.maximum_walk_distance = 0.002
        
        self.calculate_location = lambda x: (MG.node[x]['location']['lat'], MG.node[x]['location']['long'])# tuple
        self.current_location = self.calculate_location(stop_id)
    
    
    def look_around(self, current_stop_id):
        '''
        Look at current stop coordinates and find close stops
        Receives a maximum distance (in degrees) and 
        returns a dictionary with close stops and distance (in meters) to each of them
        
        maximum_dist:  str, int
        return: dict {stop_id: walk time from self.stop_id} 
        '''
        close_stops = {}
        
        long_max = self.MG.node[current_stop_id]['location']['long'] + self.maximum_walk_distance
        long_min = self.MG.node[current_stop_id]['location']['long'] - self.maximum_walk_distance
        
        lat_max = self.MG.node[current_stop_id]['location']['lat'] + self.maximum_walk_distance
        lat_min = self.MG.node[current_stop_id]['location']['lat'] - self.maximum_walk_distance
        
        for stops in self.MG.node.keys():
            
            if (self.MG.node[stops]['location']['long']<=long_max and
                self.MG.node[stops]['location']['long']>=long_min and
                self.MG.node[stops]['location']['lat']<=lat_max and
                self.MG.node[stops]['location']['lat']>=lat_min):
                
                walk_distance = (get_distance(
                    self.MG.node[current_stop_id]['location']['long'],
                    self.MG.node[current_stop_id]['location']['lat'],
                    self.MG.node[stops]['location']['long'],
                    self.MG.node[stops]['location']['lat']) / self.walking_speed)
                
                walk_time = walk_distance / self.walking_speed
                
                close_stops[stops] = walk_time
                
                

        return(close_stops)
    
    def wait_time(self, current_time, connection_timetable):
        
        wait_time = []
        
        connection_timetable.sort()

        #print(connection_timetable)
        for x in connection_timetable: 
            
            diff = x - current_time
            
            if diff >= 0:
                return diff
        
        return None
    
    def dijkstra(self):
        
        priority_list = pd.DataFrame(columns=['travel_time', 'current_time', 'stop_id_from', 'stop_id_to', 'line_id']) 
        
        visited_list = pd.DataFrame(columns=['travel_time',  'current_time', 'stop_id_from', 'stop_id_to', 'line_id']) 
        
        current_stop_id = self.stop_id
        
        current_time = self.start_time
        
        walkable_neighbours = self.look_around(current_stop_id)
        
        i = 0
        for stop_id, walk_time in walkable_neighbours.items():
            visited_list.loc[i] = [0, 0, stop_id, stop_id, None]
            i = i + 1
        
        i = 0
    
        while 1:
            #print()
            #print(i)
            #print(current_stop_id)
            #print(visited_list)
            
            # get walkable neighbours
            
            walkable_neighbours = self.look_around(current_stop_id)
            #print('current time:', current_time)
            for stop_id, walk_time in walkable_neighbours.items():
                #print('current_stop_id, stop_id, initial_time')
                #print(current_stop_id, stop_id, initial_time)
            
                # get time travel for all neighbours
                try:
                    self.MG.node[stop_id]['viagens'].items()
                except:
                    print(stop_id)
                for conn_neigh_key, conn_neigh_value in self.MG.node[stop_id]['viagens'].items():
                    #print(conn_neigh_key)
                    
                    # get stop_id_to
                    output = getpath(self.MG.edge[stop_id], conn_neigh_key)
                    if output == None:
                        continue
                    stop_id_to, line_id, *rest = output
                    
                    # if stop_id_to in visited_list, don't calculate time
                    if stop_id_to in visited_list:
                        continue

                    # get next wait time, if not a bus, continue
                    
                    wait_time =  self.wait_time(current_time, conn_neigh_value)
                    if wait_time == None:
                        continue
                    # get transport time
                    transport_time = self.MG.edge[stop_id][stop_id_to][line_id]['attr']['tempo']
                    #print('wait_time, transport_time, initial_time')
                    #print(wait_time, transport_time, walk_time)
                    # get total travel time (transport_time + wait_time + departure time)
                    travel_time = (transport_time + wait_time + walk_time)

                    # if not in priority, and priority is not greater, add to priority
                    if stop_id_to in visited_list['stop_id_to'].tolist():
                        continue

                    elif stop_id_to not in priority_list['stop_id_to'].tolist():

                        if len(priority_list) == 0:

                            priority_list.loc[0] = [travel_time,
                                                    current_time + travel_time,
                                                    stop_id,
                                                    stop_id_to,
                                                    conn_neigh_key]
                        else:

                            priority_list.loc[priority_list.tail(1).index[0] + 1] = [travel_time,
                                                                                     current_time + travel_time,
                                                                                stop_id,
                                                                                stop_id_to,
                                                                                conn_neigh_key]

                    elif priority_list[priority_list['stop_id_to'] == stop_id_to]['travel_time'].tolist()[0] >= travel_time:

                        priority_list.loc[priority_list[priority_list['stop_id_to'] == stop_id_to].index[0]] = [travel_time, 
                                                                                                                current_time + travel_time,
                                                                                                                 stop_id,
                                                                                                                 stop_id_to, 
                                                                                                                conn_neigh_key]

                
            
            # sort priority, get next stop_id in priority_list if traveled time < total_time_travel 
            
            #print('AQUI',pd.merge(priority_list, visited_list, how='inner', on=['stop_id_to']))
            
            priority_list.sort_values(by='current_time', inplace=True)
            
            priority_list.reset_index(inplace=True, drop=True)
            
            current_stop_time = priority_list.iloc[0]['current_time']
            i = i+1

            if (current_stop_time - self.start_time < self.total_time_travel ):

                current_stop_id = priority_list.iloc[0]['stop_id_to']

                visited_list = visited_list.append(priority_list.iloc[0])
                #print(priority_list)

                priority_list.drop([0], inplace=True)
                
                current_time = current_stop_time
            
            else:
                print('aqui')
                print(i)
                break
                
                
            if i > 1000:
                break
            
        self.visited_set = visited_list

In [ ]:
node_id = '18866'
current_time = 12
walker = Walker(MG, node_id, travel_time=30, current_time=current_time)

%time walker.dijkstra()

#fig = walker.plotly(walker.visited_set, walker.MG)
#py.iplot(fig, filename='Multiple Mapbox', validate=False)

In [ ]:
walker.mapbox(walker.visited_set, walker.MG)

In [79]:
ls

Calculate_Isochrone.ipynb  gtfs_to_graph.ipynb
Joao_Solution.ipynb        map.html
README.md                  osm.html
Testes Guilherme.ipynb     requirements.txt
Untitled.ipynb             scripts/
data/                      to_SQL/
graphs/


In [119]:
lista = [[0,'aa'], ['0', 'bb']]
html = '['

for i, l in enumerate(lista):

            if i == 0:
                html = html + '[{}, {}]'.format(l[0], l[1]) 
            else:
                html = html + ',[{}, {}]'.format(l[0], l[1]) 
html

'[[0, aa],[0, bb]'